# 04 - Création d'un dataset des données nettoyées de job_events (après fractionnement)

Ce notebook génère 1 csv :

- clean_merge_job_events_dataset.csv

Etapes :

- Suppression des lignes avec les valeurs timestamp_start manquantes

- Suppression des colonnes ayant des valeurs uniques

- Remplacement des données maquantes

- Conversion des types des séries en fonction des types des valeurs

# A. Imports

## a) Librairies

In [1]:
import os, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

## b) Données

In [2]:
# source path to raw metrics dataset
filename = 'raw_merge_job_events_dataset.csv'
path = '../data/jobs'
job_events = os.path.join(path, filename)
# target path to save merge raw metrics dataset
save_csv = '../data/jobs/clean_merge_job_events_dataset.csv'

# B. Dataframe

## a) Création

In [3]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(job_events, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37412 entries, 0 to 37411
Data columns (total 48 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   jobId                                                  37412 non-null  int64  
 1   timestamp_start                                        37398 non-null  object 
 2   totalCopies_start                                      37398 non-null  float64
 3   jsonVersion_x                                          22919 non-null  float64
 4   LED_iper                                               37398 non-null  float64
 5   bars_iper                                              37398 non-null  object 
 6   drops_iper                                             37398 non-null  float64
 7   dithering_iper                                         37398 non-null  object 
 8   deadPixelsOffset_iper                         

C:\Users\Allan\AppData\Local\Temp\ipykernel_15692\3017163755.py:2: DtypeWarning: Columns (8,13,14,18,24,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(job_events, index_col=0)


In [4]:
# réindexation
df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df.head(5)

,jobId,timestamp_start,totalCopies_start,jsonVersion_x,LED_iper,bars_iper,drops_iper,dithering_iper,deadPixelsOffset_iper,level_user,...,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,timestamp_end,totalCopies_end,consumption_operatorSideTanks_varnishConsumption,path,image,jsonVersion_y
0,1645522997,2022-02-22T09:43:18.1166478Z,6.0,NaN,50.0,"[1, 2]",4.0,False,0.0,Operator,...,25.0,False,3.0,SUCCESS,2022-02-22T09:44:33.3894028Z,6.0,4.585923,D:/IMAGES/Standard/1504750#1/0000001.tif,/9j/4AAQSkZJRgABAQEASABIAAD/4gxYSUNDX1BST0ZJTE...,NaN
1,1645523101,2022-02-22T09:45:01.3041033Z,11.0,NaN,50.0,"[1, 2]",4.0,False,0.0,Operator,...,25.0,False,3.0,SUCCESS,2022-02-22T09:46:34.9290928Z,11.0,2.917403,D:/IMAGES/Standard/1504749#1/0000001.tif,/9j/4AAQSkZJRgABAQEASABIAAD/4gxYSUNDX1BST0ZJTE...,NaN
2,1645523250,2022-02-22T09:47:30.3197334Z,7.0,NaN,50.0,"[1, 2]",4.0,False,0.0,Operator,...,25.0,False,3.0,SUCCESS,2022-02-22T09:48:37.5548877Z,7.0,0.423666,D:/IMAGES/Standard/1505959#1/0000001 V01.tif,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBw...,NaN
3,1645523396,2022-02-22T09:49:56.2985885Z,11.0,NaN,50.0,"[1, 2]",4.0,False,0.0,Operator,...,25.0,False,3.0,SUCCESS,2022-02-22T09:51:14.4060997Z,11.0,1.100145,D:/IMAGES/Standard/1505734#1/0000001 V01.tif,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBw...,NaN
4,1645523577,2022-02-22T09:52:57.3050451Z,47.0,NaN,50.0,"[1, 2]",4.0,False,0.0,Operator,...,25.0,False,3.0,SUCCESS,2022-02-22T09:55:59.9936088Z,47.0,4.701610,D:/IMAGES/Standard/1505736#1/0000001 V01.tif,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBw...,NaN


## b) Nettoyage

### 1. Suppression de lignes

In [5]:
# on cherche les index des lignes sans timestamp de départ
no_start_row = list(df[df['timestamp_start'].isna()].index)
no_start_row

[37398,
 37399,
 37400,
 37401,
 37402,
 37403,
 37404,
 37405,
 37406,
 37407,
 37408,
 37409,
 37410,
 37411]

In [6]:
# on supprime les lignes sans timestamp de départ
df.drop(no_start_row, axis=0, inplace=True)

### 2. Suppression de colonnes

In [7]:
# liste des colonnes
df.columns

Index(['jobId', 'timestamp_start', 'totalCopies_start', 'jsonVersion_x',
       'LED_iper', 'bars_iper', 'drops_iper', 'dithering_iper',
       'deadPixelsOffset_iper', 'level_user', 'operator_user', 'speed_ifoil',
       'enabled_ifoil', 'optifoil_ifoil', 'vacuumIn_ifoil', 'vacuumOut_ifoil',
       'stampAreas_ifoil', 'heater1Enabled_ifoil', 'speedTensionIn_ifoil',
       'speedTensionOut_ifoil', 'heater1Temperature_ifoil',
       'x_imageLayout_layout', 'y_imageLayout_layout',
       'name_paperFormat_layout', 'width_paperFormat_layout',
       'height_paperFormat_layout', 'speed_layout', 'power_irDryers',
       'power_uvDryers', 'redScore_gridMode_remoteScannerRegistration',
       'redScore_cropmarksMode_remoteScannerRegistration',
       'x_cropmark1_cropmarksMode_remoteScannerRegistration',
       'y_cropmark1_cropmarksMode_remoteScannerRegistration',
       'x_cropmark2_cropmarksMode_remoteScannerRegistration',
       'y_cropmark2_cropmarksMode_remoteScannerRegistration',
     

In [8]:
# liste des types des colonnes
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [9]:
# suppression des colonnes non pertinentes
col_to_drop = [
    'path', 
    'image']
df.drop(col_to_drop, axis=1, inplace=True)

### 3. Données maquantes

In [10]:
# on liste les colonnes qui contiennent des valeurs nulles
col_with_nan = []
for col in df.columns:
    if df[col].isnull().any():
        col_with_nan.append(col)
        print(f'{col} : {df[col].unique()}') if df[col].nunique() <= 10 else print(f'{col} : {df[col].nunique()} - dtype : {df[col].dtype}')     

jsonVersion_x : [nan  2.]
name_paperFormat_layout : [nan 'B1' 'A4']
jobState : ['SUCCESS' 'ERROR' 'CANCELED' nan]
timestamp_end : 37319 - dtype : object
totalCopies_end : 175 - dtype : float64
consumption_operatorSideTanks_varnishConsumption : 34218 - dtype : float64
jsonVersion_y : [nan  2.]


In [11]:
# on remplace les 'name_paperFormat' sans nom par UNDEFINED
df['name_paperFormat_layout'] = df['name_paperFormat_layout'].fillna('UNDEFINED')

In [12]:
# on remplace les 'jobState' non défini par UNDEFINED
df['jobState'] = df['jobState'].fillna('UNDEFINED')

In [13]:
# on converti les valeur de 'timestamp_end' en booléen
#df['timestamp_end'] = df['timestamp_end'].apply(lambda x: False if pd.isna(x) else True)
df['timestamp_end'] = df['timestamp_end'].fillna(df['timestamp_start'])

In [14]:
# on remplace les valeurs de 'totalCopies_end' nulles par 0
df['totalCopies_end'] = df['totalCopies_end'].fillna(float(0))

In [15]:
# on remplace les valeurs de 'varnishConsumption' nulles par 0
df['consumption_operatorSideTanks_varnishConsumption'] = df['consumption_operatorSideTanks_varnishConsumption'].fillna(float(0))

### 4. Conversion de type

In [16]:
# conversion des colonnes de type 'float' dont toutes les valeurs ont des décimales à 0 en type 'int'
for col in list(df.select_dtypes(exclude=['object','bool']).columns):
    if not True in (math.modf(value)[0] != float(0) for value in list(df[col].unique())):
        df[col] = df[col].astype('int64')

In [17]:
# on liste les valeurs (si pas plus de 5) des colonnes de type object
for col in list(df.select_dtypes(include=['object']).columns):
    print(f"{col} : {df[col].unique()}") if df[col].nunique() <= 5  else print(f"{col} : {df[col].nunique()} values")

timestamp_start : 37398 values
bars_iper : ['[1, 2]' '[2]' '[1]']
dithering_iper : [False True]
level_user : ['Operator' 'Distributor']
operator_user : ['User' 'Distributor']
enabled_ifoil : [True False]
optifoil_ifoil : [False True]
stampAreas_ifoil : 26 values
heater1Enabled_ifoil : [True False]
name_paperFormat_layout : ['UNDEFINED' 'B1' 'A4']
enable_specialSubstrate_remoteScannerRegistration : [False True]
jobState : ['SUCCESS' 'ERROR' 'CANCELED' 'UNDEFINED']
timestamp_end : 37398 values


In [18]:
# on peut convertir certaines colonnes au format datetime
df['timestamp_start'] = pd.to_datetime(df['timestamp_start'], utc=True)
df['timestamp_end'] = pd.to_datetime(df['timestamp_end'], utc=True)

In [19]:
# on peut convertir le type des colonnes contenant True au format booléen
for col in list(df.select_dtypes(include=['object']).columns):
    values = list(df[col].unique())
    if True in (len(values) == 2  and value == True for value in values):
        df[col] = df[col].astype('bool')

In [20]:
# on liste les valeurs (si pas plus de 5) des colonnes de type object
for col in list(df.select_dtypes(include=['object']).columns):
    print(f"{col} : {df[col].unique()}") if df[col].nunique() <= 5  else print(f"{col} : {df[col].nunique()} values")

bars_iper : ['[1, 2]' '[2]' '[1]']
level_user : ['Operator' 'Distributor']
operator_user : ['User' 'Distributor']
stampAreas_ifoil : 26 values
name_paperFormat_layout : ['UNDEFINED' 'B1' 'A4']
jobState : ['SUCCESS' 'ERROR' 'CANCELED' 'UNDEFINED']


### 5. Output csv

In [21]:
df.columns

Index(['jobId', 'timestamp_start', 'totalCopies_start', 'jsonVersion_x',
       'LED_iper', 'bars_iper', 'drops_iper', 'dithering_iper',
       'deadPixelsOffset_iper', 'level_user', 'operator_user', 'speed_ifoil',
       'enabled_ifoil', 'optifoil_ifoil', 'vacuumIn_ifoil', 'vacuumOut_ifoil',
       'stampAreas_ifoil', 'heater1Enabled_ifoil', 'speedTensionIn_ifoil',
       'speedTensionOut_ifoil', 'heater1Temperature_ifoil',
       'x_imageLayout_layout', 'y_imageLayout_layout',
       'name_paperFormat_layout', 'width_paperFormat_layout',
       'height_paperFormat_layout', 'speed_layout', 'power_irDryers',
       'power_uvDryers', 'redScore_gridMode_remoteScannerRegistration',
       'redScore_cropmarksMode_remoteScannerRegistration',
       'x_cropmark1_cropmarksMode_remoteScannerRegistration',
       'y_cropmark1_cropmarksMode_remoteScannerRegistration',
       'x_cropmark2_cropmarksMode_remoteScannerRegistration',
       'y_cropmark2_cropmarksMode_remoteScannerRegistration',
     

In [22]:
df.to_csv(path_or_buf=Path(save_csv))